In [ ]:
from fastai.vision.widgets import *
from random import uniform
import numpy as np
from scipy.stats import norm,norminvgauss
import math  
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
def frho(iPD):
    rho = .12 * (1 - np.exp(-50 * iPD) / (1 - np.exp(-50))) + .24 * (1 - (1 - np.exp(-50 * iPD) / (1 - np.exp(-50))))
    return rho

In [ ]:
def fLossDistSim(iPD, lgd, loan, cntgn,NoS):

    global df
    x = np.random.rand(1, NoS)
    rho = frho(iPD)
# VaR999 - value at risk 99.9% valor esperado al 99.9
    var999 = lgd * norm.cdf((np.sqrt(1 / (1 - rho))) * norm.ppf(iPD, 0, 1) + (np.sqrt(rho / (1 - rho))) *norm.ppf(0.999, 0, 1), 0, 1)
# simulated loans LGD * 
    L = lgd * norm.cdf((np.sqrt(1 / (1 - rho))) * norm.ppf(iPD, 0, 1) + (np.sqrt(rho / (1 - rho))) * norm.ppf(x, 0, 1), 0, 1)
  
    
    L1 = L * loan
    L0 = L1 - var999 * loan
    L3 = var999 * loan *lgd
    L2 = iPD * lgd * loan
    formula7=L3-L2
    ##total loan pd Avg LGD
    ##Formula 7 Capital
#     L0 = L1 - var999 * loan 
    
    #Expected Loss
# #     plt.hist(L.flatten(), bins=2000)
# #     plt.show()
#     #Cuanto puedes perder fuera del tail Expected Value
#     ind=np.transpose(np.where(L>var999,L,0))
#     ind=ind[(ind>0)]
# #     plt.hist(ind.flatten(), bins=2000)
# #     plt.show()
    F=formula7
    df=df.assign(F=F)
    df.to_csv ("t.csv", index = False, header=True)

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
btn_run = widgets.Button(description='Clasificar')
btn_run

In [ ]:
def on_click_classify(change):
    with open("CSVTEST.csv", "w+b") as i:
      i.write(btn_upload.data[0])
    df = pd.read_csv("CSVTEST.csv", low_memory=False)
    t=df.shape[0]
    lgd = np.array(df["lgd"]).reshape((t,1))
    loan = np.array(df["loan"]).reshape((t,1))
    cntgn = np.array(df["cntgn"]).reshape((t,1))
    iPD = np.array(df["iPD"]).reshape((t,1))
    NoS = 1000000
    fLossDistSim(iPD, lgd, loan, cntgn,NoS)

btn_run.on_click(on_click_classify)

In [ ]:
#hide_output
VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run])